### Imports

In [1]:
import pandas as pd
import numpy as np

### Load Data

In [183]:
df = pd.read_csv('data/laptop_price_wf.csv', encoding = 'latin-1')
df.head()

,Unnamed: 0,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Price_euros,Weight_fl
0,0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1339.69,1.37
1,1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,898.94,1.34
2,2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,575.00,1.86
3,3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,2537.45,1.83
4,4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1803.60,1.37


### Feature Engineering

In [184]:
# product
print(df['Product'].unique())
print(len(df['Product'].unique()))

# makes sense to drop this column as there are 618 unique values out of 1303 values.

['MacBook Pro' 'Macbook Air' '250 G6' 'Aspire 3' 'ZenBook UX430UN'
 'Swift 3' 'Inspiron 3567' 'MacBook 12"' 'IdeaPad 320-15IKB' 'XPS 13'
 'Vivobook E200HA' 'Legion Y520-15IKBN' '255 G6' 'Inspiron 5379'
 '15-BS101nv (i7-8550U/8GB/256GB/FHD/W10)' 'MacBook Air' 'Inspiron 5570'
 'Latitude 5590' 'ProBook 470' 'LapBook 15.6"'
 'E402WA-GA010T (E2-6110/2GB/32GB/W10)'
 '17-ak001nv (A6-9220/4GB/500GB/Radeon' 'IdeaPad 120S-14IAP'
 'Inspiron 5770' 'ProBook 450' 'X540UA-DM186 (i3-6006U/4GB/1TB/FHD/Linux)'
 'Inspiron 7577' 'X542UQ-GO005 (i5-7200U/8GB/1TB/GeForce'
 'Aspire A515-51G' 'Inspiron 7773' 'IdeaPad 320-15ISK' 'Rog Strix'
 'X751NV-TY001T (N4200/4GB/1TB/GeForce' 'Yoga Book' 'ProBook 430'
 'Inspiron 3576' '15-bs002nv (i3-6006U/4GB/128GB/FHD/W10)' 'VivoBook Max'
 'GS73VR 7RG' 'X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux)' 'Vostro 5471'
 'IdeaPad 520S-14IKB' 'UX410UA-GV350T (i5-8250U/8GB/256GB/FHD/W10)'
 'ZenBook Pro' 'Stream 14-AX040wm' 'V310-15ISK (i5-7200U/4GB/1TB/FHD/W10)'
 'FX753VE-GC093 (i7

In [185]:
# screen resolution
# Indication of IPS/VA/TN panels not present in all rows.
# Only keep information on touchscreen and resolution.
df['Touchscreen'] = np.where(df['ScreenResolution'].str.contains('Touchscreen', case = False, na = False), 1, 0)

# drop everything else but resolution
df['ScreenResolution'] = df.apply(lambda x: x['ScreenResolution'][-8:], axis = 1)
# drop whitespace
df['ScreenResolution'] = df['ScreenResolution'].str.replace(' ', '')

In [186]:
# cpu
# performance and price of CPUs are similar in the same "series"
# removing 'core'
df['Cpu'] = df['Cpu'].str.replace('Core', '')
# keeping the first 3 "words" to reduce unique values
df['Cpu'] = df['Cpu'].str.split().str[:2].str.join(sep = ' ')

# creating new column for cpu brand and dropping it from cpu column
df['Cpu_brand'] = df['Cpu'].str.split().str[0]
df['Cpu'] = df['Cpu'].str.split().str[1:].str.join(sep = ' ')

In [187]:
# memory
# creating column for presence of ssd
df['ssd'] = np.where(df['Memory'].str.contains('ssd|hybrid', case = False, na = False), 1, 0)

# changing 1tb to 1024gb, 2tb to 2048gb, dropping gb
df['Memory'] = df['Memory'].str.replace('1.0', '1', regex = False)
df['Memory'] = df['Memory'].str.replace('1TB', '1024')
df['Memory'] = df['Memory'].str.replace('2TB', '2048')
df['Memory'] = df['Memory'].str.replace('GB', '')

# keeping memory size only (add if more than one)
df['Memory'] = df['Memory'].str.split().apply(lambda x: [a for a in x if a.isdigit()])
df['Memory'] = df['Memory'].apply(lambda x: [int(a) for a in x])
df['Memory'] = df['Memory'].apply(lambda x: sum(x))

In [188]:
# gpu
# performance and price of each unique GPU is similar in the same "series" for laptops
# keeping the first 3 "words" to reduce unique values
df['Gpu'] = df['Gpu'].str.split().str[:2].str.join(sep = ' ')

# creating new column for gpu brand and dropping it from gpu column
df['Gpu_brand'] = df['Gpu'].str.split().str[0]
df['Gpu'] = df['Gpu'].str.split().str[1:].str.join(sep = ' ')

In [189]:
# dropping rows with values of frequency < 5
counts_col2 = df.groupby('Cpu')['Cpu'].transform(len)
counts_col3 = df.groupby('Gpu')['Gpu'].transform(len)

mask = (counts_col2 > 3) & (counts_col3 > 3)

df[mask]

,Unnamed: 0,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Price_euros,Weight_fl,Touchscreen,Cpu_brand,ssd,Gpu_brand
0,0,1,Apple,MacBook Pro,Ultrabook,13.3,560x1600,i5,8GB,128,Iris,macOS,1339.69,1.37,0,Intel,1,Intel
1,1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,i5,8GB,128,HD,macOS,898.94,1.34,0,Intel,0,Intel
2,2,3,HP,250 G6,Notebook,15.6,920x1080,i5,8GB,256,HD,No OS,575.00,1.86,0,Intel,1,Intel
3,3,4,Apple,MacBook Pro,Ultrabook,15.4,880x1800,i7,16GB,512,Radeon,macOS,2537.45,1.83,0,Intel,1,AMD
4,4,5,Apple,MacBook Pro,Ultrabook,13.3,560x1600,i5,8GB,256,Iris,macOS,1803.60,1.37,0,Intel,1,Intel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,920x1080,i7,4GB,128,HD,Windows 10,638.00,1.80,1,Intel,1,Intel
1299,1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,200x1800,i7,16GB,512,HD,Windows 10,1499.00,1.30,1,Intel,1,Intel
1300,1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Celeron,2GB,64,HD,Windows 10,229.00,1.50,0,Intel,0,Intel
1301,1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,i7,6GB,1024,Radeon,Windows 10,764.00,2.19,0,Intel,0,AMD


In [190]:
df['Cpu'].unique()

array(['i5', 'i7', 'A9-Series', 'i3', 'M', 'Atom', 'E-Series',
       'A6-Series', 'Celeron', 'Ryzen', 'Pentium', 'FX', 'Xeon',
       'A10-Series', 'A8-Series', 'A12-Series', 'A4-Series', 'Cortex'],
      dtype=object)